In [17]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm
import numpy as np

In [33]:
MODEL_PATH = "yolo11s_1024px_final/weights/best.pt"
VIDEO_PATH = "../datasets/videos/merged.mp4"
CSV_OUTPUT_PATH = "../data/results/ball_metrics.csv"

In [34]:
df = pd.DataFrame()

In [35]:
model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), "Failed to open video"

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

ball_rows = []
frame_idx = 0

with tqdm(total=total_frames, desc="Detecting football") as pbar:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        h, w, _ = frame.shape

        results = model.predict(
            source=frame,
            imgsz=1024,
            conf=0.15,
            iou=0.5,
            device=0,
            verbose=False
        )

        ball_detected = False
        ball_x = ball_y = ball_w = ball_h = ball_conf = np.nan

        if results and results[0].boxes is not None:
            boxes = results[0].boxes
            if len(boxes) > 0:
                idx = boxes.conf.argmax().item()
                x1, y1, x2, y2 = boxes.xyxy[idx].cpu().numpy()
                conf = boxes.conf[idx].item()

                ball_detected = True
                ball_x = ((x1 + x2) / 2) / w
                ball_y = ((y1 + y2) / 2) / h
                ball_w = (x2 - x1) / w
                ball_h = (y2 - y1) / h
                ball_conf = conf

        ball_rows.append({
            "frame": frame_idx,
            "ball_detected": ball_detected,
            "ball_x": ball_x,
            "ball_y": ball_y,
            "ball_w": ball_w,
            "ball_h": ball_h,
            "ball_confidence": ball_conf
        })

        frame_idx += 1
        pbar.update(1)

cap.release()

df_ball = pd.DataFrame(ball_rows)
df_ball.to_csv(CSV_OUTPUT_PATH, index=False)

print("Ball-only CSV saved to:", CSV_OUTPUT_PATH)
print("Total frames:", len(df_ball))


Detecting football: 100%|██████████| 832/832 [00:32<00:00, 25.80it/s]

Ball-only CSV saved to: ../data/results/ball_metrics.csv
Total frames: 832


In [36]:
df_ball

,frame,ball_detected,ball_x,ball_y,ball_w,ball_h,ball_confidence
0,0,False,NaN,NaN,NaN,NaN,NaN
1,1,False,NaN,NaN,NaN,NaN,NaN
2,2,False,NaN,NaN,NaN,NaN,NaN
3,3,False,NaN,NaN,NaN,NaN,NaN
4,4,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
827,827,False,NaN,NaN,NaN,NaN,NaN
828,828,False,NaN,NaN,NaN,NaN,NaN
829,829,False,NaN,NaN,NaN,NaN,NaN
830,830,False,NaN,NaN,NaN,NaN,NaN


In [37]:
print(df_ball.columns)

Index(['frame', 'ball_detected', 'ball_x', 'ball_y', 'ball_w', 'ball_h',
       'ball_confidence'],
      dtype='object')


In [38]:
df_pose = pd.read_csv("../data/results/pose_metrics.csv")

In [39]:
df_pose

,frame,timestamp_ms,pose_detected,interpolated,missing_streak,lm_0_x,lm_0_y,lm_0_z,lm_0_vis,lm_1_x,...,lm_30_z,lm_30_vis,lm_31_x,lm_31_y,lm_31_z,lm_31_vis,lm_32_x,lm_32_y,lm_32_z,lm_32_vis
0,0,0,True,False,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,33,True,False,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,66,True,False,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100,True,False,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,133,True,False,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,827,27587,True,False,0,0.628520,0.419963,0.044203,0.998557,0.627729,...,-0.038644,0.974759,0.608531,0.547110,-0.021069,0.879745,0.629468,0.553818,-0.044750,0.909206
828,828,27620,True,False,0,0.629940,0.419058,0.047784,0.998615,0.629248,...,-0.051068,0.970895,0.613048,0.545286,-0.018527,0.833838,0.627592,0.553787,-0.058212,0.896098
829,829,27653,True,False,0,0.630597,0.416132,0.038643,0.998057,0.630019,...,-0.045638,0.966006,0.620762,0.545423,-0.017412,0.814728,0.626466,0.553655,-0.052939,0.881530
830,830,27687,True,False,0,0.633074,0.412091,0.047119,0.998179,0.632470,...,-0.039678,0.955003,0.630259,0.545866,-0.002786,0.784444,0.626680,0.552765,-0.046173,0.854953


In [40]:
print(len(df_pose) == len(df_ball))
print(df_pose["frame"].equals(df_ball["frame"]))


True
True


In [41]:
df_combined = df_pose.merge(
    df_ball,
    on="frame",
    how="inner"
)

In [42]:
OUTPUT_CSV = "../data/results/pose_plus_ball.csv"
df_combined.to_csv(OUTPUT_CSV, index=False)

print("Saved combined CSV to:", OUTPUT_CSV)

Saved combined CSV to: ../data/results/pose_plus_ball.csv


In [45]:
for col in df_combined.columns:
    print(col)

frame
timestamp_ms
pose_detected
interpolated
missing_streak
lm_0_x
lm_0_y
lm_0_z
lm_0_vis
lm_1_x
lm_1_y
lm_1_z
lm_1_vis
lm_2_x
lm_2_y
lm_2_z
lm_2_vis
lm_3_x
lm_3_y
lm_3_z
lm_3_vis
lm_4_x
lm_4_y
lm_4_z
lm_4_vis
lm_5_x
lm_5_y
lm_5_z
lm_5_vis
lm_6_x
lm_6_y
lm_6_z
lm_6_vis
lm_7_x
lm_7_y
lm_7_z
lm_7_vis
lm_8_x
lm_8_y
lm_8_z
lm_8_vis
lm_9_x
lm_9_y
lm_9_z
lm_9_vis
lm_10_x
lm_10_y
lm_10_z
lm_10_vis
lm_11_x
lm_11_y
lm_11_z
lm_11_vis
lm_12_x
lm_12_y
lm_12_z
lm_12_vis
lm_13_x
lm_13_y
lm_13_z
lm_13_vis
lm_14_x
lm_14_y
lm_14_z
lm_14_vis
lm_15_x
lm_15_y
lm_15_z
lm_15_vis
lm_16_x
lm_16_y
lm_16_z
lm_16_vis
lm_17_x
lm_17_y
lm_17_z
lm_17_vis
lm_18_x
lm_18_y
lm_18_z
lm_18_vis
lm_19_x
lm_19_y
lm_19_z
lm_19_vis
lm_20_x
lm_20_y
lm_20_z
lm_20_vis
lm_21_x
lm_21_y
lm_21_z
lm_21_vis
lm_22_x
lm_22_y
lm_22_z
lm_22_vis
lm_23_x
lm_23_y
lm_23_z
lm_23_vis
lm_24_x
lm_24_y
lm_24_z
lm_24_vis
lm_25_x
lm_25_y
lm_25_z
lm_25_vis
lm_26_x
lm_26_y
lm_26_z
lm_26_vis
lm_27_x
lm_27_y
lm_27_z
lm_27_vis
lm_28_x
lm_28_y
lm_28_z
lm_

In [46]:
df_mismatch = df_combined[
    df_combined["pose_detected"] ^ df_combined["ball_detected"]
]



In [47]:
df_mismatch

,frame,timestamp_ms,pose_detected,interpolated,missing_streak,lm_0_x,lm_0_y,lm_0_z,lm_0_vis,lm_1_x,...,lm_32_x,lm_32_y,lm_32_z,lm_32_vis,ball_detected,ball_x,ball_y,ball_w,ball_h,ball_confidence
0,0,0,True,False,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
1,1,33,True,False,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
2,2,66,True,False,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
3,3,100,True,False,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
4,4,133,True,False,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,827,27587,True,False,0,0.628520,0.419963,0.044203,0.998557,0.627729,...,0.629468,0.553818,-0.044750,0.909206,False,NaN,NaN,NaN,NaN,NaN
828,828,27620,True,False,0,0.629940,0.419058,0.047784,0.998615,0.629248,...,0.627592,0.553787,-0.058212,0.896098,False,NaN,NaN,NaN,NaN,NaN
829,829,27653,True,False,0,0.630597,0.416132,0.038643,0.998057,0.630019,...,0.626466,0.553655,-0.052939,0.881530,False,NaN,NaN,NaN,NaN,NaN
830,830,27687,True,False,0,0.633074,0.412091,0.047119,0.998179,0.632470,...,0.626680,0.552765,-0.046173,0.854953,False,NaN,NaN,NaN,NaN,NaN


In [48]:
409 frames

SyntaxError: invalid syntax (1157760594.py, line 1)